In [26]:
import pandas as pd
import pdfplumber
import pandasql as psql
import re
from prettytable import PrettyTable



In [ ]:
def extraer_texto_pdf(ruta_pdf):
    texto_completo = ""
    
    with pdfplumber.open(ruta_pdf) as pdf:
        for page in pdf.pages:
            texto_completo += page.extract_text()
    
    return texto_completo


In [27]:
def encolumnar_datos(texto):
    """
    Esta función recibe una cadena de texto con datos en formato no estructurado y los convierte en columnas.
    
    :param texto: Cadena de texto con datos no estructurados.
    :return: DataFrame de pandas con los datos encolumnados.
    """
    # Separar cada línea del texto
    lineas = texto.strip().split('\n')
    
    # Crear una lista para almacenar las filas procesadas
    filas = []
    
    # Procesar cada línea para separar en columnas
    for linea in lineas:
        # Usar una expresión regular para capturar las columnas, con la columna Cuotas opcional
        patron = r'(\d{2}-\d{2}-\d{2})\s+([*K]?\s?[A-Z].+?)(?:\s+(\d{2}/\d{2}))?\s+(\d{6})\s+([\d.,]+)'
        coincidencia = re.match(patron, linea)
        
        if coincidencia:
            fecha, descripcion, cuotas, comprobante, monto = coincidencia.groups()
            # Si no hay información en Cuotas, dejar el campo vacío
            cuotas = cuotas if cuotas else ''
            filas.append([fecha, descripcion.strip(), cuotas, comprobante, monto])
    
    # Crear un DataFrame con las filas procesadas
    df = pd.DataFrame(filas, columns=['Fecha', 'Descripcion', 'Cuotas', 'Comprobante', 'Monto'])
    
    return df


In [38]:
def concat_df():
    summary_list =['resumen_enero.pdf','resumen_febrero.pdf','resumen_marzo.pdf','resumen_abril.pdf','resumen_mayo.pdf','resumen_junio.pdf','resumen_julio.pdf','resumen_agosto.pdf','resumen_septiembre.pdf','resumen_octubre.pdf','resumen_noviembre.pdf','resumen_diciembre.pdf'] 

In [28]:
def consumption_analysis(df):

    # Funcion para identificar cantidad de consumos en un mismo local:
    
    # Contar la frecuencia de cada valor en la columna 'Descripción'
    conteo = df['Descripcion'].value_counts()
    
    # Mapear el conteo de frecuencias a la columna del DataFrame
    df['q_repetido'] = df['Descripcion'].map(conteo)
    
    #------------------------------------------------------------------------------------
    # Funcion para identificar consumos duplicados:
    
    df['concat_desc_monto'] = df['Fecha'] + '' + df['Descripcion'] + ' ' + df['Monto']
    
    # Contar la frecuencia de cada valor en la columna 'Descripción'
    conteo_consum_dupli = df['concat_desc_monto'].value_counts()
    
    # Mapear el conteo de frecuencias a la columna del DataFrame
    df['consumo_duplicado'] = df['concat_desc_monto'].map(conteo_consum_dupli) 

    return df

In [29]:
# Construccion de categorias:

def apply_categories (df):

    categorias = {
        '* TIENDA DE CAFE': 'salida',
        '* RAPPI ARG S.A.S.': 'comida',
        '* MOLINA PANADERIA': 'salida',
        '* DIA TIENDA 409': 'comida',
        '* FARMACITY': 'medicina',
        'P MICROSOFT*1 MES Z62PJZACGU5Q115776': 'suscripcion',
        '* MEDICUS SA DEB A 12688472000': 'obra social',
        '* RES BOYACA': 'comida',
        'V LinkedIn Pre P38 LinkedIn Pre P3821': 'suscripcion',
        '* CARREFOUR EXPRESS AV CARA': 'comida',
        '* PERSONAL FLOW 100329902571001': 'servicios',
        'K HAVANNA SA': 'salidas',
        '* ALMACEN DE PIZZAS': 'salidas',
        '* EDESUR DEB AUT 056340211': 'servicios',
        '* MULTIPASTA': 'comida',
        '* RAPPI': 'comida',
        'K MERPAGO*SPORTCLU': 'gimansio',
        'P MICROSOFT*1 MES Z62FLR7HJIKF115776': 'suscripcion',
        '* LA TROPILLA CARNICERIA ': 'comida',
        '* CAFE REGISTRADO': 'salida',
        '* OTAWA DELI S.A.': 'comida',
        '* DLO*RAPPI.': 'comida',
        '* RUIZ LA PASTELERIA': 'comida',
        '* FOGOSA': 'salida',
        '* LA BUENOS AIRES PASTEL': 'comida',
        '* EASY CABALLITO': 'hogar',
        '* QUOTIDIANO-QUOTIDIANO': 'salida',
        '* KUKE SRL - AXION': 'auto',
        'K MERPAGO*PSA': 'hogar',
        '* SARKANYSOFIA ALCORTA': 'hogar',
        '* FIERA': 'salida',
        '* NFS ARCOS': 'salida'}

    df['categoria'] = df['Descripcion'].map(categorias)

    return df

In [30]:
def visual_style(df):
# Formato visual para la tabla de salda

# Setup de opciones para la visualizacion de los DF correctamente en Visual Studio

    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_colwidth', None)
    pd.set_option('display.max_columns', None)

# Prety Table Format:

    df_new_style = PrettyTable()
    df_new_style.field_names = df.columns.tolist()
    for row in df.itertuples(index=False):
         df_new_style.add_row(row)

    return(df_new_style)     



***Main***

In [31]:
# Se ejecuta la funcion de lectura del PDF y traduccion a DF:
def main(file_name):
    texto = extraer_texto_pdf('resumen_septiembre.pdf')   # Lectura del PDF
    df_sumary_card = (encolumnar_datos(texto))            # Transformacion del PDF en DF
    df_sumary_card = consumption_analysis(df_sumary_card) # Construye las agregaciones necesarias para el analisis
    df_sumary_card = apply_categories (df_sumary_card)    # Se generan las categorias que clasifican los consumos
    df_sumary_card = visual_style(df_sumary_card)

    return df_sumary_card

# **Test**

In [35]:
texto = extraer_texto_pdf('resumen_septiembre.pdf')   # Lectura del PDF
df_sumary_card = (encolumnar_datos(texto))            # Transformacion del PDF en DF
#df_sumary_card = consumption_analysis(df_sumary_card) # Construye las agregaciones necesarias para el analisis
#df_sumary_card = apply_categories (df_sumary_card)    # Se generan las categorias que clasifican los consumos
#df_sumary_card = visual_style(df_sumary_card)

print(df_sumary_card)

       Fecha                            Descripcion Cuotas Comprobante  \
0   23-06-24                    K MERPAGO*TCTIENDAS  03/06      222697   
1   04-07-24                          K MERPAGO*PSA  03/06      212596   
2   09-08-24                 K MERPAGO*MERCADOLIBRE  02/06      143241   
3   22-08-24         * MEDICUS SA DEB A 12688472000             545063   
4   23-08-24                     K MERPAGO*SPORTCLU             474832   
5   23-08-24                                * RAPPI             004033   
6   24-08-24   P MICROSOFT*1 MES Z62JNQ1XVP8B115776             074008   
7   24-08-24                * QUOTIDIANO-QUOTIDIANO             838912   
8   24-08-24                     * KUKE SRL - AXION             001656   
9   24-08-24                                * FIERA             000385   
10  25-08-24                   K MERPAGO*GOOPYSTORE             148977   
11  26-08-24  V LinkedIn Pre P77 LinkedIn Pre P7767             857802   
12  26-08-24                          